In [8]:
## Arxiv--Research
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [9]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [10]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [11]:
pip install arxiv

Note: you may need to restart the kernel to use updated packages.


In [12]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [13]:
tools=[wiki,arxiv]

In [34]:
##custom tools[RAG tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [35]:
from langchain_community.document_loaders import WebBaseLoader

In [38]:
pip install faiss-cpu

  Using cached faiss_cpu-1.11.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
Using cached faiss_cpu-1.11.0-cp312-cp312-win_amd64.whl (15.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [39]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000029AD7AEB050>, search_kwargs={})

In [40]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

In [41]:
tools=[wiki,arxiv,retriever_tool]

In [42]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\searchengine\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000029AF816E980>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000029AD7AEB050>, search_kwargs=

In [44]:
pip install langchain_groq

  Using cached groq-0.29.0-py3-none-any.whl.metadata (16 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached groq-0.29.0-py3-none-any.whl (130 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)

   ---------------------------------------- 0/3 [distro]
   ------------- -------------------------- 1/3 [groq]
   ------------- -------------------------- 1/3 [groq]
   ------------- -------------------------- 1/3 [groq]
   ------------- -------------------------- 1/3 [groq]
   ------------- -------------------------- 1/3 [groq]
   ------------- -------------------------- 1/3 [groq]
   ------------- -------------------------- 1/3 [groq]
   ------------- -------------------------- 1/3 [groq]
   -------------------------- ------------- 2/3 [langchain_groq]
   ---------------------------------------- 3/3 [langchain_groq]

Note: you may need to restart the kernel to use updated packages.


In [48]:
##Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
# import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")
# openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [49]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [50]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [51]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [52]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Get started by creating your first prompt.
Iterate on models and prompts using the Playground.
Manage prompts programmatically in your application.
Was this page helpful?You can leave detailed feedback on GitHub.NextQuick StartObservabilityEvalsPrompt EngineeringCommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.

Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet

{'input': 'Tell me about Langsmith',
 'output': "Based on the output from the tool call, it seems that Langsmith is a platform for building production-grade LLM (Large Language Model) applications. It provides features for observability, evaluation, and prompt engineering to help developers ship their applications quickly and with confidence.\n\nLangsmith offers several key features, including:\n\n1. Observability: Langsmith allows developers to analyze traces and configure metrics, dashboards, and alerts to monitor their application's performance.\n2. Evals: Langsmith enables developers to evaluate their application over production traffic and score its performance, as well as receive human feedback on their data.\n3. Prompt Engineering: Langsmith provides automatic version control and collaboration features for iterating on prompts.\n\nLangsmith is also framework-agnostic, meaning it can be used with or without Langchain's open-source frameworks, Langchain and Langgraph.\n\nLangsmith

In [53]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witBased on the result from the "wikipedia" tool, I can provide a more detailed response.

Machine learning is a fascinating field that has become increasingly important in recent years. It involves developing algorithms that can learn from data and improve their performance over time. This is in contrast to traditional programming, where the rules and logic are hardcoded.

There are several types of machine learning, including supervised and unsupervised learning. Supervised learning involves training an algorithm on labeled data, where the correct output is already known. The algorithm learns to predict the output for new, unseen data. Unsup

{'input': 'What is machine learning',
 'output': 'Based on the result from the "wikipedia" tool, I can provide a more detailed response.\n\nMachine learning is a fascinating field that has become increasingly important in recent years. It involves developing algorithms that can learn from data and improve their performance over time. This is in contrast to traditional programming, where the rules and logic are hardcoded.\n\nThere are several types of machine learning, including supervised and unsupervised learning. Supervised learning involves training an algorithm on labeled data, where the correct output is already known. The algorithm learns to predict the output for new, unseen data. Unsupervised learning involves training an algorithm on unlabeled data, and it must find patterns or structure in the data on its own.\n\nSome common applications of machine learning include image and speech recognition, natural language processing, and recommender systems. It\'s also used in self-driv

In [54]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cIt appears that the paper "1706.03762" is actually titled "Attention Is All You Need" and it's about sequence transduction models.

> Finished chain.


{'input': "What's the paper 1706.03762 about?",
 'output': 'It appears that the paper "1706.03762" is actually titled "Attention Is All You Need" and it\'s about sequence transduction models.'}